In [1]:
import pyaudio
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf

/home/pi/.local/lib/python3.7/site-packages/numba/core/errors.py:144: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/home/pi/.local/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/pi/.local/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
#TF LITE MODEL LOADING
interpreter = tf.lite.Interpreter(model_path="mel_smote_v1_iter_1.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# input shape to ensure correct dimension
input_shape = input_details[0]['shape']

In [3]:
#PREPARE MIC SETTINGS
p = pyaudio.PyAudio()
for ii in range(p.get_device_count()):
    print(p.get_device_info_by_index(ii).get('name'))
    if p.get_device_info_by_index(ii).get('name') == 'iRig Mic Cast HD: USB Audio (hw:1,0)':
        index = ii
print('Mic found at',index)

bcm2835 HDMI 1: - (hw:0,0)
iRig Mic Cast HD: USB Audio (hw:1,0)
sysdefault
lavrate
samplerate
speexrate
pulse
upmix
vdownmix
dmix
default
Mic found at 1


In [4]:
#PRE PROCESS AUDIO AND RUN MODEL
def process(final_arr, start_time, fig):
    #PreProcess
    spec = librosa.feature.melspectrogram(y=final_arr, sr=sr, n_fft = int(window_width*sr), hop_length =int(sliding*sr), fmax=sr/2)
    #Convert amplitude to decibels
    db_spec = librosa.power_to_db(spec, ref=np.max)
    ax = plt.axes()
    librosa.display.specshow(db_spec, sr=sr, hop_length =int(sliding*sr),fmax=sr/2)
    ax.axis('off')
    fig.tight_layout(pad=0)
    ax.margins(0)
    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    image_from_plot = np.float32(image_from_plot) #For Model Compatibility
    image_from_plot = np.expand_dims(image_from_plot, axis = 0)
    plt.clf()
    #Record total preprocessing time
    pre_proc_time.append(time.time() - start) 
    #End PreProcess
    
    #Call TF LITE Model
    start_eval = time.time()
    interpreter.set_tensor(input_details[0]['index'], image_from_plot)
    interpreter.invoke()  # runs the test
    output_data = interpreter.get_tensor(output_details[0]['index'])
    #Record model inference time
    infer_time.append(time.time() - start_eval)

In [5]:
form_1 = pyaudio.paInt16 # 24-bit resolution but using 16
chans = 1 # 1 channel
samp_rate = 44100 # 44.1kHz sampling rate
chunk = 4096 # 2^12 samples for buffer
record_secs = 3 # seconds to record
dev_index = index # device index found by p.get_device_info_by_index(ii)

In [6]:
# MEL CONFIGS
window_width =  0.025 #25 ms  window size 
sliding = 0.01 #10ms stride 
sr = samp_rate

fig = plt.figure(figsize=(1.70, 1.12), dpi=100)

<Figure size 170x112 with 0 Axes>

In [7]:
pre_proc_time = [] #Record prepreoccsing times
infer_time = []

In [8]:
audio = pyaudio.PyAudio() # create pyaudio instantiation

# create pyaudio stream
stream = audio.open(format = form_1,rate = samp_rate,
                    channels = chans, 
                    input_device_index = dev_index,
                    input = True,
                    frames_per_buffer=chunk)
#Repeat for 10 minutes (200 samples 3*200 = 600 seconds)
for iteration in range(200):
    frames = []

    # loop through stream and append audio chunks to frame array
    for ii in range(0,int((samp_rate/chunk)*record_secs)):
        data = stream.read(chunk,exception_on_overflow = False)
        frames.append(np.fromstring(data,dtype=np.float16))

    #Begin processing
    start = time.time()
    final_arr = np.array(frames).ravel()
    np.nan_to_num(final_arr,copy=False)
    process(final_arr, start, fig)
# stop the stream, close it, and terminate the pyaudio instantiation
stream.stop_stream()
stream.close()
audio.terminate()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  app.launch_new_instance()


<Figure size 432x288 with 0 Axes>

In [9]:
ppt = np.array(pre_proc_time)
print('mean preprocessing time',ppt.mean())
print('std',ppt.std())

mean preprocessing time 0.22687649488449096
std 0.019475593118999986


In [10]:
it = np.array(infer_time)
print('mean inferance time',it.mean())
print('std',it.std())

mean inferance time 0.018013337850570677
std 0.0021230191626599388


In [11]:
total_time = it+ ppt

print('mean total time',total_time.mean())
print('std',total_time.std())

mean total time 0.24488983273506165
std 0.01971057829559917
